<a href="https://colab.research.google.com/github/shazzad-hasan/training-reproducable-deep-learning-models/blob/main/LeNet-5/lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/shazzad-hasan/training-reproducable-deep-learning-models.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/training-reproducable-deep-learning-models/LeNet-5

In [ ]:
!ls

In [ ]:
# import required libraries
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import local helper functions
from helper_dataset import dataloader_mnist
from helper_train import train
from helper_evaluate import compute_loss_accuracy, set_all_seeds
from helper_plot import show_examples, show_sample_test_result

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available")

if train_on_gpu:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
# settings
random_seed = 123
valid_size = 0.2
batch_size = 32
num_epochs = 20

set_all_seeds(random_seed)

In [ ]:
from torchvision import transforms

data_transform  = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,),(0.5,))]
    
)

train_loader, valid_loader, test_loader, classes = dataloader_mnist(
    batch_size = batch_size, 
    train_transform = data_transform,
    test_transform = data_transform,
    valid_size = valid_size
)


In [ ]:
print(classes)
num_class = len(classes)

In [ ]:
# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

In [ ]:
import torch.nn as nn

class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super().__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        in_channels = 1 if self.grayscale else 3

        self.features = torch.nn.Sequential(nn.Conv2d(in_channels, 6, 5),
                                            nn.Tanh(),
                                            nn.MaxPool2d(2),
                                            nn.Conv2d(6, 16, 5),
                                            nn.Tanh(),
                                            nn.MaxPool2d(2))

        self.classifier = torch.nn.Sequential(nn.Linear(16*5*5, 120),
                                              nn.Tanh(),
                                              nn.Linear(120, 84),
                                              nn.Tanh(),
                                              nn.Linear(84, num_classes))
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

model = LeNet5(num_classes=10, grayscale=True)
# move model to the right device
model.to(device)

print(model)

In [ ]:
import torch.optim as optim

# specify loss
criterion = nn.CrossEntropyLoss() # categorical cross-entropy loss

params = model.parameters()
optimizer = optim.SGD(params, lr=0.01)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, mode="min")

In [ ]:
train_losses, valid_losses = train(model, num_epochs, train_loader, valid_loader,
                                   test_loader, optimizer, criterion, device) 